## Visual insights into Passwords: Part 2

### Objective
In part 1, we took a look the dataset, and had a few visualizations. Next, we want to conduct topic modelling from a machine learning point of view. More specifically, we will employ a [fastText embedding](https://en.wikipedia.org/wiki/FastText) to represent passwords as 300-dimensional vectors. To visualize them in 2 or 3 dimensions, we will use dimension reduction techniques. 

In [3]:
# import the necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from gensim.models import fasttext

In [3]:
df = pd.read_csv('../data/data2use/USA2/data0.csv', index_col=0)
df.head(10)

,password,frequency,distance_score,passlength,unique_c,first_char,last_char,number_of_uppercase,number_of_digits,number_of_symbols,number_of_lowercase,category,zxcvbn
0,123456,67329,1.00,6,6,digit,digit,0,6,0,0,numeric,0
1,123456789,25745,1.00,9,9,digit,digit,0,9,0,0,numeric,0
2,qwerty,25539,1.00,6,6,lower,lower,0,0,0,6,alphabetic,0
3,password,11259,3.50,8,7,lower,lower,0,0,0,8,alphabetic,0
4,12345,9922,1.00,5,5,digit,digit,0,5,0,0,numeric,0
5,b123456,9150,1.54,7,7,lower,digit,0,6,0,1,numeric,1
6,123456b,9143,1.43,7,7,digit,lower,0,6,0,1,numeric,1
7,123456c,8251,1.67,7,7,digit,lower,0,6,0,1,numeric,1
8,c123456,8244,1.36,7,7,lower,digit,0,6,0,1,numeric,1
9,12345678,8088,1.00,8,8,digit,digit,0,8,0,0,numeric,0


For our current purpose, we mostly only need the password column of the dataset. 

### Password embedding with fastText
Word embedding is a technique used in Natural Language Processing (NLP) to represent words as points in a high-dimensional space, thus having associated numerical values. The most basic desired property of a word embedding is that words with similar meaning should be close to each other. 

fastText is a library for word embedding created by Facebook's AI Research lab. A word embedding has a vocabulary that it is built upon **----to be continued**

In [4]:
# loading fasttext model
# model = fasttext.load_facebook_vectors('../data/fasttext_models/wiki.en.bin')
model = fasttext.load_facebook_vectors('../data/fasttext_models/wiki.en.bin')

KeyboardInterrupt: 

In [4]:
# example of an embedding password
model['hello123']

array([-0.12791799, -0.06494759, -0.04391411, -0.09344378, -0.20545131,
       -0.07863272,  0.16797353, -0.17578395, -0.05386898, -0.15918167,
       -0.18677713,  0.05414825,  0.07892729, -0.26773155,  0.240542  ,
       -0.26164737,  0.0308893 ,  0.01671401,  0.08233713, -0.14278439,
       -0.00716576,  0.3266556 ,  0.07943831,  0.07748769, -0.15551227,
       -0.1852229 , -0.15285002, -0.12167276, -0.05522198,  0.2579628 ,
       -0.3626395 ,  0.15684427, -0.20550492,  0.13036738, -0.16333723,
       -0.04185367, -0.24040319, -0.05731153, -0.11681544, -0.01761103,
        0.17473377, -0.09494759,  0.06681579,  0.08521983, -0.02771697,
        0.19598943, -0.07935197,  0.03730717, -0.38345242,  0.01025466,
       -0.09711715, -0.4918841 ,  0.16596034,  0.02118551,  0.04988703,
        0.08059371, -0.00439768, -0.28616515,  0.14371178,  0.23619308,
        0.10012786,  0.26847646, -0.09483156, -0.26119354,  0.04041209,
        0.03156268,  0.14464356,  0.27149758, -0.16569377,  0.05

Note that this is a 300-dimensional vectors. Next, we embed our list of passwords. For our current purpose, we will only look at the top 100000 most frequent passwords in the dataset. 

In [43]:
# Create an empty dataframe to store the embedding vectors
emb = pd.DataFrame(columns=range(300), index=df.password[:100000])

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
password,,,,,,,,,,,,,,,,,,,,,
123456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123456789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
qwerty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
password,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
basar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lady09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
for index, row in emb.iterrows():
    emb.loc[index] = model[index]
emb

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
password,,,,,,,,,,,,,,,,,,,,,
123456,0.085685,-0.016403,0.262378,-0.107177,-0.042068,-0.130793,0.256913,-0.405635,0.066068,0.064177,...,-0.418605,0.210022,0.277876,-0.458475,0.006005,0.060775,0.061683,0.188673,0.146644,0.022421
123456789,-0.130535,0.068342,-0.033046,-0.054806,-0.170985,-0.065777,0.07449,-0.504951,-0.024171,0.082977,...,-0.263262,0.124102,0.025722,-0.242219,-0.065629,-0.118434,0.019818,0.197971,-0.08601,0.218956
qwerty,-0.031566,0.385277,-0.021763,-0.158742,-0.056417,-0.410767,-0.074155,0.140993,-0.004023,-0.354872,...,0.035786,-0.100373,0.122964,-0.317977,0.2453,-0.303113,-0.169563,0.180374,0.180517,0.329809
password,-0.028614,0.364398,0.130883,0.084824,-0.198433,-0.259452,0.128979,0.207979,-0.242296,-0.024767,...,-0.023839,-0.223112,-0.006987,-0.489869,0.150709,0.126917,-0.65894,0.299658,0.263977,0.469383
12345,0.116955,0.016777,0.212105,0.021843,-0.304988,-0.005762,0.063454,-0.609076,0.212839,0.097452,...,-0.267434,0.035151,0.29801,-0.407719,-0.0522,0.087453,-0.025644,0.120497,0.060088,-0.024097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290199,-0.078659,-0.013763,-0.017855,0.10139,-0.16413,0.051178,-0.068467,0.119254,-0.273734,0.077733,...,-0.116509,-0.044367,0.009115,-0.121545,0.313241,0.594927,0.171317,0.223865,0.334095,-0.125014
basar,-0.16095,0.211703,0.008398,0.049956,0.120377,0.110713,0.47223,-0.01877,0.430658,0.113599,...,-0.892448,0.044026,0.174957,-0.09596,0.096383,-0.093946,-0.47397,-0.037964,-0.062385,0.034707
lady09,0.168224,0.206265,-0.055783,0.113901,0.202915,-0.033973,0.129309,0.069552,0.26972,0.461483,...,0.198738,-0.008041,0.079289,-0.065166,0.19471,-0.058503,-0.091192,-0.132704,0.205788,0.12344


In [42]:
emb1.to_csv('../data/data2use/Embedding/emb1.csv')

That was simple enough. It is important to keep in mind that each row of the dataframe contains the vector representation of the corresponding password. Next, we perform dimension reduction on these vectors. Some well-known dimension reduction techniques are PCA and [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding). Another reduction technique is [UMAP](https://github.com/lmcinnes/umap). 

### Topic modelling via Clustering & Plotly visualization

In [130]:
frame = pd.read_csv('../data/data2use/USA2/data0.csv',index_col=0)
# df.drop([df.columns[0]],inplace=True,axis=1)
frame

,frequency,distance_score,passlength,unique_c,first_char,last_char,number_of_uppercase,number_of_digits,number_of_symbols,number_of_lowercase,category,zxcvbn,number_of_specials,passpolicy
password,,,,,,,,,,,,,,
123456,67329,1.00,6,6,digit,digit,0,6,0,0,numeric,0,0,False
123456789,25745,1.00,9,9,digit,digit,0,9,0,0,numeric,0,0,False
qwerty,25539,1.00,6,6,lower,lower,0,0,0,6,alphabetic,0,0,False
password,11259,3.50,8,7,lower,lower,0,0,0,8,alphabetic,0,0,False
12345,9922,1.00,5,5,digit,digit,0,5,0,0,numeric,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
compikuls,1,3.30,9,9,lower,lower,0,0,0,9,alphabetic,3,0,False
shahroozofficia,1,4.63,15,9,lower,lower,0,0,0,15,alphabetic,4,0,False
milkymoo,1,1.71,8,6,lower,lower,0,0,0,8,alphabetic,2,0,False


In [131]:
emb = pd.read_csv('../data/data2use/Embedding/emb.csv', index_col=0)
emb

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
password,,,,,,,,,,,,,,,,,,,,,
123456,0.085685,-0.016403,0.262378,-0.107177,-0.042068,-0.130793,0.256913,-0.405635,0.066068,0.064177,...,-0.418605,0.210022,0.277876,-0.458475,0.006005,0.060775,0.061683,0.188673,0.146644,0.022421
123456789,-0.130535,0.068342,-0.033046,-0.054806,-0.170985,-0.065777,0.074490,-0.504951,-0.024171,0.082977,...,-0.263262,0.124102,0.025722,-0.242219,-0.065629,-0.118434,0.019818,0.197971,-0.086010,0.218956
qwerty,-0.031566,0.385277,-0.021763,-0.158742,-0.056417,-0.410767,-0.074155,0.140993,-0.004023,-0.354872,...,0.035786,-0.100373,0.122964,-0.317977,0.245300,-0.303113,-0.169563,0.180374,0.180517,0.329809
password,-0.028614,0.364398,0.130883,0.084824,-0.198433,-0.259452,0.128979,0.207979,-0.242296,-0.024767,...,-0.023839,-0.223112,-0.006987,-0.489869,0.150709,0.126917,-0.658940,0.299658,0.263977,0.469383
12345,0.116955,0.016777,0.212105,0.021843,-0.304988,-0.005762,0.063454,-0.609076,0.212839,0.097452,...,-0.267434,0.035151,0.298010,-0.407719,-0.052200,0.087453,-0.025644,0.120497,0.060088,-0.024097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290199,-0.078659,-0.013763,-0.017855,0.101390,-0.164130,0.051178,-0.068467,0.119254,-0.273734,0.077733,...,-0.116509,-0.044367,0.009115,-0.121545,0.313241,0.594927,0.171317,0.223865,0.334095,-0.125014
basar,-0.160950,0.211703,0.008398,0.049956,0.120377,0.110713,0.472230,-0.018770,0.430658,0.113599,...,-0.892448,0.044026,0.174957,-0.095960,0.096383,-0.093946,-0.473970,-0.037964,-0.062385,0.034707
lady09,0.168224,0.206265,-0.055783,0.113901,0.202915,-0.033973,0.129309,0.069552,0.269720,0.461483,...,0.198738,-0.008041,0.079289,-0.065166,0.194710,-0.058503,-0.091192,-0.132704,0.205788,0.123440


In [105]:
def filter_reduce(df,X,thres, clus_algo, num_cluster, kind, dim=2):
    """
    Input: password frequency cutoff, algorithm, number of clusters (if applicable),
            dimension reduction map, reduction dimension
    Output: dataframe containing the embedding and other useful features
    """
    
    df = df[df.frequency >= thres]
    X = X.iloc[:len(df), :]
    
    # apply standardization
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), index=X.index)
    
    # fitting cluster algo
    if clus_algo == 'kmeans':
        clustering = KMeans(n_clusters=num_cluster, n_init=20)
        clustering.fit(X)
    elif clus_algo == 'dbscan':
        clustering = DBSCAN(eps=16,min_samples=3)
        clustering.fit(X)
    
    # reduction 
    if kind=='umap':
        reducer = umap.UMAP(n_components=dim)
    elif kind=='tsne':
        reducer = TSNE(n_components=dim, init='pca')
#     centroids = pd.DataFrame(kmean.cluster_centers_, columns=X.columns)
#     X = pd.concat([centroids,X], axis=0)
#     k = len(centroids)
    idx = X.index
    X = reducer.fit_transform(X)
    if dim==2:
        X = pd.DataFrame(X,index=idx, columns=['x','y'])
    elif dim==3:
        X = pd.DataFrame(X,index=idx, columns=['x','y','z'])
    
    # add label column
    X['label'] = clustering.labels_
    X['label'] = X['label'].astype('string')
#     X.iloc[:k,2] = [i for i in range(k)]
#     X.iloc[k:,2] = kmean.labels_
#     X['label'] = X['label'].astype('int32')
    
    # concatenate X and df
    X = pd.concat([X,df], axis=1)
    return X

#### k-means clustering
A popular approach for clustering is through k-means. The approach works as follows. Given a positive integer $k$, the k-means algorithm initializes $k$ "random" points, then gradually moves these points around to become cluster centroids. 

This approach has a few challenges. First, it is fairly sensitive to initilialization. Second, it can be hard to decide on what $k$ is. 

In [23]:
# example: password frquency >= 100, kmeans, 10 clusters, tsne reduction map
kmeans = filter_reduce(df,X,100,'kmeans',10,'tsne',2)
kmeans.reset_index(level=0,inplace=True)
px.scatter(data_frame=kmeans, x='x', y='y', color='label', hover_data=['password'])

Essentially, k-means is supposed to minimize the intra-cluster variations , more precisely, the sum of squares of distances from each point to the corresponding centroid (in the language of analysis of variance, SSE). A heuristic approach to choose the number of clusters  the *elbow method*. 

In [116]:
def elbow(frame, emb, thres, c):
    distortions = []
    c_range = [ i for i in range(5,c+1)]
    
    for i in c_range:
        df = frame.copy()
        X = emb.copy()
        
        df = df[df.frequency >= thres]
        X = X.iloc[:len(df), :]
        
        # apply standardization
        scaler = StandardScaler()
        X = pd.DataFrame(scaler.fit_transform(X), index=X.index)
        
        kmeans = KMeans(n_clusters=i, n_init=20)
        kmeans.fit(X)
        distort = np.sum(kmeans.inertia_)
        distortions.append(distort)
    fig = px.scatter( x = c_range, y=distortions)
    fig.show()
    return

In [117]:
elbow(df, X, 100, 30)

In [118]:
elbow(df, X, 50, 30)

As we can see, in our situation, it is a little hard to decide what our number of cluster should be. I would say 14, but you might disagree, it is subjective after all. Moreover, we can see a few staircase steps, making things even trickier. This happens since our data is quite high-dimensional, and quite noisy.

Another method that we can employ is through the *silhouette coefficient*. 

In [123]:
def silhouette(frame, emb, thres, c):
    sil_scores = []
    c_range = [ i for i in range(5,c+1)]
    for i in c_range:
        df = frame.copy()
        X = emb.copy()
        
        df = df[df.frequency >= thres]
        X = X.iloc[:len(df), :]
        
        # apply standardization
        scaler = StandardScaler()
        X = pd.DataFrame(scaler.fit_transform(X), index=X.index)
        
        kmeans = KMeans(n_clusters=i, n_init=20)
        kmeans.fit(X)
        labels = kmeans.labels_
        silhouette = silhouette_score(X,labels)
        sil_scores.append(silhouette)
    fig = px.scatter( x = c_range, y=sil_scores)
    fig.show()
    return

In [135]:
silhouette(frame, emb, 100, 50)

This is quite strange, I don't quite understand this behaviour yet. 

In [55]:
# example: password frquency >= 50, kmeans, 14 clusters, visualized with tsne reduction map
kmeans = filter_reduce(df,X,50,'kmeans',14,'tsne',2)
kmeans.reset_index(level=0,inplace=True)
px.scatter(data_frame=kmeans, x='x', y='y', color='label', hover_data=['password'],
          color_discrete_sequence=px.colors.qualitative.Alphabet)

Going through the labels, we see that the clusters have the following (general) behaviours:
- cluster 0: passwords contain mostly regular sequences of numbers such as 123456.
- cluster 1: This cluster is a little noisy. It contains some much smaller cluster of regular number sequences such as 11111, 22222, 123123. However, it contains a large noisy cluster consisting of fairly random sequence of numbers, and sequences of uppercase characters.
- cluster 2: This cluster appears to be even noisier than cluster 1. Most of the passwords here are quite random, and do not seem to follow any particular pattern. 
- cluster 3: This cluster is actually quite nice. It is a sport-themed cluster. It contains names of sports (baseball, basketball1, soccer, football, cricket, bowling, swimming), names of US states, sport club names (fenerbahce, besiktas, bayern, borussia, shalke, chivas, rangers, indians, etc) 
- cluster 4 mostly contains certain groups of non-English names: Portuguese/Spanish names, Russian names.
- cluster 5 contains certain other groups of non-English names: Indian, Middle Eastern.
- cluster 6: a few offensive passwords, passwords indicating emotions such as *iloveyou*, *ihateyou*, or characteristics such as *cute*, *sexy*. 
- cluster 7 is a little noisy. However, at the center, we see passwords on food (juice, sugar, carrot, peach, apple, etc). Most passwords here are actually English words. 
- cluster 8 contains mostly female names common in the US. It also contains some smaller related clusters (mother, sister, queen, princess, duchess) 
- cluster 9 contains words related to gaming and to certain popular cultures. For example: game (minecraft, nintendo, playstation), popular culture (gundam, inuyasha, naruto, gandalf, legolas, wolverine, bumblebee, superman), adventure descriptive (knight, dragon, darkness, oblivion, wizard, warlock), etc.  
- cluster 10 is again quite noisy. It contains actual English words, but many of these are quite vague to categorize (origin, system, running, spectrum). It also contains various smaller and denser clusters: military (marine, commando, infantry), places (canada, montreal, toronto), among many others. 
- cluster 11 contains English male names (steve, gordon, chris, mike), English last names (thompson, campbell, smith, miller).
- cluster 12 might be one of the most regular cluster, containing almost exclusively Chinese (sounding) words.
- cluster 13 is also a very interesting cluster. The vast majority of its clusters are popular passwords such as qwerty, password, asdfgh, abcde. It also contains certain clusters related to electronics (laptop, compputer, admin, calculator, compaq, logitech, motorola).

Next, let us look at a bigger portion of the dataset, with frequency>=10. 

In [60]:
elbow(df, X, 10, 30)

In [68]:
df[df.frequency>=10]

,password,frequency,distance_score,passlength,unique_c,first_char,last_char,number_of_uppercase,number_of_digits,number_of_symbols,number_of_lowercase,category,zxcvbn,number_of_specials,passpolicy
0,123456,67329,1.00,6,6,digit,digit,0,6,0,0,numeric,0,0,False
1,123456789,25745,1.00,9,9,digit,digit,0,9,0,0,numeric,0,0,False
2,qwerty,25539,1.00,6,6,lower,lower,0,0,0,6,alphabetic,0,0,False
3,password,11259,3.50,8,7,lower,lower,0,0,0,8,alphabetic,0,0,False
4,12345,9922,1.00,5,5,digit,digit,0,5,0,0,numeric,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69870,111111qwerty123,10,0.90,15,9,digit,digit,0,9,0,6,mixed,1,0,False
69871,02121983,10,3.57,8,6,digit,digit,0,8,0,0,numeric,1,0,False
69872,goats,10,4.87,5,5,lower,lower,0,0,0,5,alphabetic,1,0,False
69873,dale08,10,5.23,6,6,lower,digit,0,2,0,4,mixed,1,0,False


In [139]:
kmeans10 = filter_reduce(df,X,100,'kmeans',12,'tsne',2)
kmeans10.reset_index(level=0,inplace=True)
kmeans10

,password,x,y,label,frequency,distance_score,passlength,unique_c,first_char,last_char,number_of_uppercase,number_of_digits,number_of_symbols,number_of_lowercase,category,zxcvbn,number_of_specials,passpolicy
0,123456,41.398834,20.500994,1,67329,1.00,6,6,digit,digit,0,6,0,0,numeric,0,0,False
1,123456789,31.565910,25.632479,1,25745,1.00,9,9,digit,digit,0,9,0,0,numeric,0,0,False
2,qwerty,7.542260,41.290203,7,25539,1.00,6,6,lower,lower,0,0,0,6,alphabetic,0,0,False
3,password,27.948633,57.343910,7,11259,3.50,8,7,lower,lower,0,0,0,8,alphabetic,0,0,False
4,12345,33.412426,19.577425,1,9922,1.00,5,5,digit,digit,0,5,0,0,numeric,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,Iloveyou,-20.368675,51.193565,5,100,2.58,8,7,upper,lower,1,0,0,7,alphabetic,0,0,False
2919,godbless,-5.570852,-13.289670,11,100,3.44,8,7,lower,lower,0,0,0,8,alphabetic,1,0,False
2920,digger,6.439693,-14.871295,11,100,2.30,6,5,lower,lower,0,0,0,6,alphabetic,0,0,False
2921,prissy,-54.172947,-32.612652,11,100,4.04,6,5,lower,lower,0,0,0,6,alphabetic,1,0,False


#### DBSCAN
The challenge of k-means is that the number of clusters has to be specified. The [DBSCAN algorithm](https://en.wikipedia.org/wiki/DBSCAN) is a clustering algorithm that does not require us to to specify the number of clusters. Other advantages of DBSCAN include being able to identified non-convex clusters, resistance to noise. A disadvatange that might be relevant to us is that DBSCAN does not work well with data having large differences in densities. 

In [ ]:
def filter_reduce(df,X,thres, clus_algo, num_cluster, kind, dim=2):
    """
    Input: password frequency cutoff, algorithm, number of clusters (if applicable),
            dimension reduction map, reduction dimension
    Output: dataframe containing the embedding and other useful features
    """
    
    df = df[df.frequency >= thres]
    X = X.iloc[:len(df), :]
    
    # apply standardization
    scaler = StandardScaler()
    X = pd.DataFrame(scaler.fit_transform(X), index=X.index)
    
    # fitting cluster algo
    if clus_algo == 'kmeans':
        clustering = KMeans(n_clusters=num_cluster, n_init=20)
        clustering.fit(X)
    elif clus_algo == 'dbscan':
        clustering = DBSCAN(eps=16,min_samples=3)
        clustering.fit(X)
    
    # reduction 
    if kind=='umap':
        reducer = umap.UMAP(n_components=dim)
    elif kind=='tsne':
        reducer = TSNE(n_components=dim, init='pca')
#     centroids = pd.DataFrame(kmean.cluster_centers_, columns=X.columns)
#     X = pd.concat([centroids,X], axis=0)
#     k = len(centroids)
    idx = X.index
    X = reducer.fit_transform(X)
    if dim==2:
        X = pd.DataFrame(X,index=idx, columns=['x','y'])
    elif dim==3:
        X = pd.DataFrame(X,index=idx, columns=['x','y','z'])
    
    # add label column
    X['label'] = clustering.labels_
    X['label'] = X['label'].astype('string')
#     X.iloc[:k,2] = [i for i in range(k)]
#     X.iloc[k:,2] = kmean.labels_
#     X['label'] = X['label'].astype('int32')
    
    # concatenate X and df
    X = pd.concat([X,df], axis=1)
    return X

In [106]:
dbscan = filter_reduce(df,X,100,'dbscan',0,'umap',2)
dbscan.reset_index(level=0,inplace=True)
dbscan

,password,x,y,label,frequency,distance_score,passlength,unique_c,first_char,last_char,number_of_uppercase,number_of_digits,number_of_symbols,number_of_lowercase,category,zxcvbn,number_of_specials,passpolicy
0,123456,-2.746215,7.974618,0,67329,1.00,6,6,digit,digit,0,6,0,0,numeric,0,0,False
1,123456789,-3.977785,7.814240,0,25745,1.00,9,9,digit,digit,0,9,0,0,numeric,0,0,False
2,qwerty,0.074004,6.844137,0,25539,1.00,6,6,lower,lower,0,0,0,6,alphabetic,0,0,False
3,password,5.033079,5.848279,0,11259,3.50,8,7,lower,lower,0,0,0,8,alphabetic,0,0,False
4,12345,-3.300941,7.351188,0,9922,1.00,5,5,digit,digit,0,5,0,0,numeric,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918,Iloveyou,2.966569,3.136989,0,100,2.58,8,7,upper,lower,1,0,0,7,alphabetic,0,0,False
2919,godbless,-0.470943,3.454696,0,100,3.44,8,7,lower,lower,0,0,0,8,alphabetic,1,0,False
2920,digger,-0.501314,3.438998,-1,100,2.30,6,5,lower,lower,0,0,0,6,alphabetic,0,0,False
2921,prissy,-1.670967,-0.230575,-1,100,4.04,6,5,lower,lower,0,0,0,6,alphabetic,1,0,False
